In [1]:
!pip install tensorflow
import pathlib
import random
import string
import tensorflow.strings as tf_strings
import tensorflow.data as tf_data
import re
from keras.layers import TextVectorization
import keras
import tensorflow as tf
from keras import layers
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


2024-06-06 10:38:56.612894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 10:38:56.613026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 10:38:56.725890: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#importing the data
#downloading, saving and extracting data
text_file =  keras.utils.get_file(
    fname = "spa-eng.zip",
    origin = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract = True,
)

#this line constructs the path to the extracted text file within the downloaded directory
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

#splits the content into lines and removes the last empty line (if any)
with open(text_file, "r") as f:
    lines = f.read().split("\n")[:-1]

#a tuple to store spanish and english data
text_pairs = []

#brings out eng and spanish sentences separated by a tab
#adds "start" and "end" to spanish sentences
#appends the text_pairs tuple
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

2638744/2638744 [==============================] - 1s 0us/step


In [3]:
random.shuffle(text_pairs)
text_pairs[:5]

[('As soon as he was left alone, he opened the letter.',
  '[start] En cuanto se quedó solo, abrió la carta. [end]'),
 ('If you want to stay, you can stay.',
  '[start] SI quieres, puedes quedarte. [end]'),
 ('I like your family.', '[start] Me agrada tu familia. [end]'),
 ('Please leave everything to me.', '[start] Déjame todo a mí. [end]'),
 ('Tom was a bookkeeper.', '[start] Tom era contable. [end]')]

In [4]:
#the number of validation samples by taking 15% of the total number of pairs 
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [5]:
# things that will be removed from the texts
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

In [6]:
#it converts input strings to lowercase
#it then removes specified characters (defined in 'strip_chars') using regular expression replacement
def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# vectorization
eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

spa_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

#training sets
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

#this is called on both vectorization layers to adapt them to the training data
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

#save the vectorization layers
eng_vectorization_config = eng_vectorization.get_config()
eng_vectorization_config.pop('standardize', None)
eng_vocab = eng_vectorization.get_vocabulary()
with open('eng_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vectorization_config, f)
    
with open('eng_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    
spa_vectorization_config = spa_vectorization.get_config()
spa_vectorization_config.pop('standardize', None)
spa_vocab = spa_vectorization.get_vocabulary()
with open('spa_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vectorization_config, f)
    
with open('spa_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vocab, f)
    
#this part applies the respective text vectorization layers and returns the input to seq2seq
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )
    
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
    

In [7]:
# taking only first BATCH of data
# the batch size is 64 and the desired sequence length is 20
for inputs,targets in train_ds.take(1):
    print(inputs["encoder_inputs"].shape)
    print(targets.shape)

(64, 20)
(64, 20)


In [8]:
#defining encoder having layers.Layer that can be used in a keras model
class TransformerEncoder(layers.Layer):
    #initializer for TransformerEncoder Class
    #kwargs is to handle named arguments that is not defined previously
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        #embedding dimension, dense layer dimension, number of attention heads i.e the number of parallel attention mechanisms in the multi-head attention layer
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
            #key_dim means dimensionality of query and key vectors
        )
        #sequence of 2 dense layer. one with relu activation and the other mapping back to embedding dimension
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
                #helps in transforming the data back to the same dimensionality as the input embeddings
            ]
        )
        #normalization layer
        self.layernorm_1 = layers.LayerNormalization()
        #applied after adding the attention output to the input
        self.layernorm_2 = layers.LayerNormalization()
        #applied after the dense projection
        #layers support masking
        self.supports_masking = True
        
    def call(self, inputs, mask=None):
        #input is typically of shape [batch_size, sequence_length, embed_dim]
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
        else:
            padding_mask = None
            
        attention_output = self.attention(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        #inputs+attention_output forms residual connection
        proj_output = self.dense_proj(proj_input)
        #residual connection, also known as a skip connection, is used in neural network architectures to address the problem of vanishing gradients
        return self.layernorm_2(proj_input + proj_output)
        #adds the result of the dense projection to the input of the projection (which has already gone through the first layer normalization)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config

# Creating a Positional Embedding
class PositionalEmbedding(layers.Layer):
    #taking sequence_length, vocab_size and embed_dim as parameters and kwargs
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        #defining token embedding
        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )
        #defining position embedding
        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )
        #assigning values
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
    def call(self, inputs):
        #length means sequence length
        length = tf.shape(inputs)[-1]
        #creates a tensor of positions starting from 0 to (length-1) with step size of 1
        positions = tf.range(start = 0, limit = length, delta = 1)
        #applying tokenization on inputs and positional embedding on positions
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # adds the token and position embeddings together, integrating both types of information into a single tensor
        return embedded_tokens + embedded_positions
    
    #masking is done to ignore padding tokens in the sequence
    def compute_mask(self, inputs, mask=None):
        if mask is not None:
            return tf.not_equal(inputs, 0)
        else:
            return None
        
    #this is essential for saving and loading models that include custom layers, ensuring that these layers maintain their configuration across different sessions
    #it retrieves the configuration dictionary of the parent class, which includes basic settings and attributes handled by the parent class
    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "sequence_length": self.sequence_length,
            "embed_dim": self.embed_dim,
        })
        return config
    
# Creating a Decoder
class TransformerDecoder(layers.Layer):
    #defining parameters: embedding dimension, latent dimension, number of attention heads
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        #he first multi-head attention layer, which typically attends to the decoder’s previous outputs
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        #he first multi-head attention layer, which typically attends to the encoder's outputs
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        #sequential model containing 2 dense layers
        self.dense_proj = keras.Sequential(
            [
                #first dense layer containing relu activation
                layers.Dense(latent_dim, activation = "relu"),
                #second dense layer projects back to embed_dim to maintain the dimension
                layers.Dense(embed_dim),
            ]
        )
        #three normalization layers for attention layer 1, attention layer 2 and the dense layer
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        #this layer supports masking, which is useful for handling variable sequence lengths and padding tokens
        self.supports_masking = True
        
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        else:
            padding_mask = None
            
        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )
        
        out_2 = self.layernorm_2(out_1 + attention_output_2)
        proj_output = self.dense_proj(out_2)
        
        return self.layernorm_3(out_2 + proj_output)
    
    #generates a causal (look-ahead) attention mask for the input sequences in a Transformer decoder
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        #creates a column vector with shape [sequence_length, 1] where each row is the sequence index.
        i = tf.range(sequence_length)[:, None]
        #creates a row vector with shape [sequence_length] where each element is the sequence index
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, tf.int32)
        mask = tf.reshape(mask,(1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.convert_to_tensor([1, 1]),
            ],
            axis = 0,
        )
        return tf.tile(mask, mult)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config

In [9]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 256
latent_dim = 2048
num_heads = 8

#Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


In [10]:
epochs = 7

transformer.summary()

transformer.compile(
    "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['encoder_inputs[0][0]']      
 tionalEmbedding)                                                                                 
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

I0000 00:00:1717670366.291558      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1302/1302 [==============================] - 92s 65ms/step - loss: 1.7382 - accuracy: 0.7470 - val_loss: 1.1615 - val_accuracy: 0.8158
Epoch 2/7
1302/1302 [==============================] - 77s 59ms/step - loss: 1.0626 - accuracy: 0.8318 - val_loss: 0.8537 - val_accuracy: 0.8579
Epoch 3/7
1302/1302 [==============================] - 77s 59ms/step - loss: 0.8575 - accuracy: 0.8581 - val_loss: 0.7680 - val_accuracy: 0.8683
Epoch 4/7
1302/1302 [==============================] - 77s 59ms/step - loss: 0.7591 - accuracy: 0.8711 - val_loss: 0.7355 - val_accuracy: 0.8751
Epoch 5/7
1302/1302 [==============================] - 77s 59ms/step - loss: 0.6971 - accuracy: 0.8799 - val_loss: 0.6987 - val_accuracy: 0.8793
Epoch 6/7
1302/1302 [==============================] - 77s 59ms/step - loss: 0.6535 - accuracy: 0.8864 - val_loss: 0.6832 - val_accuracy: 0.8832
Epoch 7/7
1302/1302 [==============================] - 77s 59ms/step - loss: 0.6209 - accuracy: 0.8914 - val_loss: 0.7121 - val_accuracy: 0.

In [11]:
transformer.save("transformer_model")

In [12]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = sequence_length

def decode_sentence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    input_sentence = input_sentence.lower()
    input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated = decode_sentence(input_sentence)
    print(f"input: {input_sentence}")
    print(f"translated: {translated}")
    print()

input: tom is absolutely terrified
translated: [start] tom es absolutamente absolutamente [end]

input: could you tell me something about yourself
translated: [start] podrías decirme algo [end]

input: tom has decided to have the operation
translated: [start] tom ha decidido a la operación [end]

input: i saw him sawing a tree
translated: [start] le vi una [UNK] [end]

input: i will teach you to program but not today
translated: [start] yo te enseñaré a pasar hoy pero hoy [end]

